# Algoritmo Naive Bayes 

## Imports

In [1]:
# importamos las librerias y bibliotecas que utilizaremos para el algoritmo naive bayes
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from pandas import DataFrame
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import f1_score
import numpy as np

## Funciones auxiliares

In [2]:
# Construcción de una función que realice el particionado completo
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

In [3]:
#Función para representar el límite de desición
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib.gridspec as gridspec

def plot_decision_boundary(X, y, clf):
    mins = X.min(axis=0) - 0.5
    maxs = X.max(axis=0) + 0.5  

    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], 71),
                     np.linspace(mins[1], maxs[1], 81))
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)
    
    gs = gridspec.GridSpec(1, 2)
    gs.update(hspace=0.8)
    
    ax = plt.subplot(gs[0])
    ax.contourf(xx, yy, Z, cmap="RdBu", alpha=0.5)
    ax.plot(X[:, 0][y==1], X[:, 1][y==1], 'r.')
    ax.plot(X[:, 0][y==0], X[:, 1][y==0], 'b.')
    
    ax = plt.subplot(gs[1])
    ax.contour(xx, yy, Z, [0.5], colors='k')
    ax.plot(X[:, 0][y==1], X[:, 1][y==1], 'r.')
    ax.plot(X[:, 0][y==0], X[:, 1][y==0], 'b.')

## Lectura del conjunto de datos

In [4]:
df = pd.read_csv('dataset-equilibrado-3.csv')

##  Visualización preliminar de la información

In [5]:
# Vista de los datos cargados
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,72918,Female,53.0,1,0,Yes,Private,Urban,62.55,30.3,Unknown,1
1,72911,Female,57.0,1,0,Yes,Private,Rural,129.54,60.9,smokes,0
2,72898,Female,67.0,1,1,Yes,Private,Rural,199.24,33.9,never smoked,1
3,72882,Male,47.0,0,0,Yes,Private,Rural,75.30,25.0,formerly smoked,0
4,72861,Female,52.0,0,0,Yes,Private,Urban,69.30,20.1,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2378,132,Female,80.0,0,0,Yes,Govt_job,Urban,84.86,NaN,Unknown,0
2379,129,Female,24.0,0,0,No,Private,Urban,97.55,26.2,never smoked,0
2380,99,Female,31.0,0,0,No,Private,Urban,108.89,52.3,Unknown,0
2381,84,Male,55.0,0,0,Yes,Private,Urban,89.17,31.5,never smoked,0


In [6]:
#Validamos la cantidad de ejemplos que posee el dataset
df["stroke"].value_counts()

0    1600
1     783
Name: stroke, dtype: int64

## Primera preparación del conjunto de datos

In [7]:
#Función para transformar las columnas categoricas a numericas
df_new = pd.get_dummies(df, columns=['gender','ever_married','work_type','Residence_type','smoking_status'])

In [8]:
#Validamos que los cambios se aplicarón correctamente
df_new

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,72918,53.0,1,0,62.55,30.3,1,1,0,0,...,0,1,0,0,0,1,1,0,0,0
1,72911,57.0,1,0,129.54,60.9,0,1,0,0,...,0,1,0,0,1,0,0,0,0,1
2,72898,67.0,1,1,199.24,33.9,1,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,72882,47.0,0,0,75.30,25.0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
4,72861,52.0,0,0,69.30,20.1,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,132,80.0,0,0,84.86,NaN,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
2379,129,24.0,0,0,97.55,26.2,0,1,0,1,...,0,1,0,0,0,1,0,0,1,0
2380,99,31.0,0,0,108.89,52.3,0,1,0,1,...,0,1,0,0,0,1,1,0,0,0
2381,84,55.0,0,0,89.17,31.5,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0


In [9]:
#Eliminamos datos que son irrelevantes y que podrian alterar el comportamiento del algoritmo
df_new = df_new.drop(["id"], axis=1)

## División del conjunto de datos

In [10]:
# Division del conjunto en los diferentes subconjuntos
train_set, val_set, test_set = train_val_test_split(df_new, stratify='stroke')

In [11]:
#Verificamos las particiones realizadas
print("Longitud del Training Set:", len(train_set))
print("Longitud del Validation Set:", len(val_set))
print("Longitud del Test Set:", len(test_set))

Longitud del Training Set: 1429
Longitud del Validation Set: 477
Longitud del Test Set: 477


In [12]:
# Conjunto de datos general
X_df = df_new.drop("stroke", axis=1)
y_df = df_new["stroke"].copy()

In [13]:
# Conjunto de datos de entrenamiento
X_train = train_set.drop("stroke", axis=1)
y_train = train_set["stroke"].copy()

In [14]:
# Conjunto de datos de validación
X_val = val_set.drop("stroke", axis=1)
y_val = val_set["stroke"].copy()

In [15]:
# Conjunto de datos de pruebas
X_test = test_set.drop("stroke", axis=1)
y_test = test_set["stroke"].copy()

## Segunda preparación del conjunto de datos

In [16]:
# Construcción de un pipeline para los atributos numéricos
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('rbst_scaler', RobustScaler()),
    ])

In [17]:
# Rellenamos los valores nulos con la mediana y scalamos los datos
df_prep = num_pipeline.fit_transform(X_df)
X_train_prep = num_pipeline.fit_transform(X_train)
X_val_prep = num_pipeline.fit_transform(X_val)
X_test_prep = num_pipeline.fit_transform(X_test)

In [18]:
# Transformamos el resultado a un DataFrame de Pandas
df_prep = pd.DataFrame(df_prep, columns=X_df.columns, index=y_df.index)
X_train_prep = pd.DataFrame(X_train_prep, columns=X_train.columns, index=y_train.index)
X_val_prep = pd.DataFrame(X_val_prep, columns=X_val.columns, index=y_val.index)
X_test_prep = pd.DataFrame(X_test_prep, columns=X_test.columns, index=y_test.index)

In [19]:
#Validamos que los cambios se aplicaron correctamente 
df_prep

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,-0.027778,1.0,0.0,-0.658383,0.240506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0
1,0.083333,1.0,0.0,0.729005,4.113924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.361111,1.0,1.0,2.172517,0.696203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.194444,0.0,0.0,-0.394325,-0.430380,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.055556,0.0,0.0,-0.518588,-1.050633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,0.722222,0.0,0.0,-0.196334,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0
2379,-0.833333,0.0,0.0,0.066480,-0.278481,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,0.0
2380,-0.638889,0.0,0.0,0.301336,3.025316,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0
2381,0.027778,0.0,0.0,-0.107073,0.392405,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,0.0


## Naive Bayes con un conjunto de datos multidimensional

In [20]:
from sklearn.naive_bayes import BernoulliNB

nb_clf = BernoulliNB(alpha=1.0e-10)
nb_clf.fit(X_train_prep, y_train)

BernoulliNB(alpha=1e-10)

In [21]:
# Predecimos para el conjunto de validación 
y_pred = nb_clf.predict(X_val_prep)

In [22]:
print('F1 score: {:.3f}'.format(f1_score(y_val, y_pred, pos_label=1)))

F1 score: 0.709


In [23]:
from sklearn.metrics import accuracy_score
print('Accuracy: {:.3f}'.format(accuracy_score(y_val, y_pred)))

Accuracy: 0.778


In [24]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, y_pred)

array([[242,  78],
       [ 28, 129]], dtype=int64)

In [25]:
# Predecimos para el conjunto de pruebas
y_pred = nb_clf.predict(X_test_prep)

In [26]:
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred, pos_label=1)))

F1 score: 0.639


In [27]:
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.744


In [28]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, y_pred)

array([[193, 127],
       [102,  55]], dtype=int64)

## Gaussian Naive Bayes con un conjunto de datos multidimensional

In [38]:
# Entrenamos un Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train_prep, y_train)

GaussianNB()

In [39]:
# Predecimos para el conjunto de validación 
y_pred = clf.predict(X_val_prep)

In [40]:
print('F1 score: {:.3f}'.format(f1_score(y_val, y_pred, pos_label=1)))

F1 score: 0.588


In [41]:
print('Accuracy: {:.3f}'.format(accuracy_score(y_val, y_pred)))

Accuracy: 0.541


In [42]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, y_pred)

array([[102, 218],
       [  1, 156]], dtype=int64)

In [43]:
# Predecimos para el conjunto de pruebas
y_pred = nb_clf.predict(X_test_prep)

In [35]:
print('F1 score: {:.3f}'.format(f1_score(y_test, y_pred, pos_label=1)))

F1 score: 0.639


In [36]:
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.744


In [37]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, y_pred)

array([[193, 127],
       [102,  55]], dtype=int64)